In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Layer,Lambda


In [2]:
forestfires=pd.read_csv(r'C:\ExcelR\forestfires.csv',encoding= 'unicode_escape')


In [3]:
forestfires.head()


,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


In [4]:
forestfires.shape


(517, 31)

In [5]:
forestfires.columns


Index(['month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind',
       'rain', 'area', 'dayfri', 'daymon', 'daysat', 'daysun', 'daythu',
       'daytue', 'daywed', 'monthapr', 'monthaug', 'monthdec', 'monthfeb',
       'monthjan', 'monthjul', 'monthjun', 'monthmar', 'monthmay', 'monthnov',
       'monthoct', 'monthsep', 'size_category'],
      dtype='object')

In [6]:
forestfires=forestfires.drop({'dayfri', 'daymon', 'daysat', 'daysun', 'daythu',
       'daytue', 'daywed', 'monthapr', 'monthaug', 'monthdec', 'monthfeb',
       'monthjan', 'monthjul', 'monthjun', 'monthmar', 'monthmay', 'monthnov',
       'monthoct', 'monthsep'}, axis=1)


In [7]:
forestfires


,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,small
...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,small


In [8]:
forestfires.isnull().sum()


month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
size_category    0
dtype: int64

In [9]:
forestfires.describe()


,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


In [10]:
print("correlation:",forestfires.corr(method='pearson'))


correlation:           FFMC       DMC        DC       ISI      temp        RH      wind  \
FFMC  1.000000  0.382619  0.330512  0.531805  0.431532 -0.300995 -0.028485   
DMC   0.382619  1.000000  0.682192  0.305128  0.469594  0.073795 -0.105342   
DC    0.330512  0.682192  1.000000  0.229154  0.496208 -0.039192 -0.203466   
ISI   0.531805  0.305128  0.229154  1.000000  0.394287 -0.132517  0.106826   
temp  0.431532  0.469594  0.496208  0.394287  1.000000 -0.527390 -0.227116   
RH   -0.300995  0.073795 -0.039192 -0.132517 -0.527390  1.000000  0.069410   
wind -0.028485 -0.105342 -0.203466  0.106826 -0.227116  0.069410  1.000000   
rain  0.056702  0.074790  0.035861  0.067668  0.069491  0.099751  0.061119   
area  0.040122  0.072994  0.049383  0.008258  0.097844 -0.075519  0.012317   

          rain      area  
FFMC  0.056702  0.040122  
DMC   0.074790  0.072994  
DC    0.035861  0.049383  
ISI   0.067668  0.008258  
temp  0.069491  0.097844  
RH    0.099751 -0.075519  
wind  0.061119  0

In [11]:
#We are going to perform label encoding since it is faster than dummy variables

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
forestfires.month= label_encoder.fit_transform(forestfires.month) 
forestfires.day= label_encoder.fit_transform(forestfires.day) 
forestfires.size_category= label_encoder.fit_transform(forestfires.size_category) 
forestfires.head()


,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category
0,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1
1,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,1
2,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,1
3,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1
4,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,1


In [12]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return(x)


In [13]:
pred = forestfires.iloc[:,0:11]
target = forestfires.iloc[:,11]


In [14]:
pred1 = norm_func(pred)


In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(pred1,target,test_size=0.3,stratify = target)


In [16]:
# create model
model = Sequential()
model.add(Dense(14, input_dim=11, kernel_initializer='uniform', activation='relu'))
model.add(Dense(12,kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
hist = model.fit(x_train, y_train, validation_split=0.33, epochs=50, batch_size=10)


Epoch 1/50
25/25 [==============================] - 1s 15ms/step - loss: 0.6900 - accuracy: 0.7427 - val_loss: 0.6856 - val_accuracy: 0.7083
Epoch 2/50
25/25 [==============================] - 0s 4ms/step - loss: 0.6766 - accuracy: 0.7427 - val_loss: 0.6686 - val_accuracy: 0.7083
Epoch 3/50
25/25 [==============================] - 0s 4ms/step - loss: 0.6455 - accuracy: 0.7427 - val_loss: 0.6369 - val_accuracy: 0.7083
Epoch 4/50
25/25 [==============================] - 0s 4ms/step - loss: 0.6011 - accuracy: 0.7427 - val_loss: 0.6099 - val_accuracy: 0.7083
Epoch 5/50
25/25 [==============================] - 0s 4ms/step - loss: 0.5767 - accuracy: 0.7427 - val_loss: 0.6072 - val_accuracy: 0.7083
Epoch 6/50
25/25 [==============================] - 0s 4ms/step - loss: 0.5691 - accuracy: 0.7427 - val_loss: 0.6100 - val_accuracy: 0.7083
Epoch 7/50
25/25 [==============================] - 0s 4ms/step - loss: 0.5687 - accuracy: 0.7427 - val_loss: 0.6106 - val_accuracy: 0.7083
Epoch 8/50
25/25 [=

In [17]:
test_loss, test_acc = model.evaluate(x_test, y_test)


5/5 [==============================] - 0s 4ms/step - loss: 0.5891 - accuracy: 0.7308


In [18]:
print(hist.history.keys())



dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [19]:
hist_df = pd.DataFrame(hist.history)
hist_df["epoch"]=hist.epoch
hist_df.tail()


,loss,accuracy,val_loss,val_accuracy,epoch
45,0.554723,0.742739,0.609733,0.708333,45
46,0.554446,0.742739,0.606013,0.708333,46
47,0.552969,0.742739,0.608322,0.708333,47
48,0.553348,0.742739,0.609473,0.708333,48
49,0.551880,0.742739,0.606818,0.708333,49
